In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix)
import joblib
import warnings
warnings.filterwarnings('ignore')

# Check for GPU availability
try:
    from xgboost import XGBClassifier
    import xgboost as xgb
    XGB_OK = True
    # Check if GPU is available
    try:
        # This will raise an error if CUDA is not available
        xgb.XGBClassifier(tree_method='gpu_hist', n_estimators=1).fit([[0]], [0])
        GPU_AVAILABLE = True
        print("✅ GPU detected and available for XGBoost")
    except Exception as e:
        GPU_AVAILABLE = False
        print(f"⚠️ GPU not available: {e}")
        print("   XGBoost will use CPU")
except ImportError:
    XGB_OK = False
    GPU_AVAILABLE = False
    print("⚠️ XGBoost not found. Install with: pip install xgboost")

print("=" * 80)
print("FLIGHT DELAY PREDICTION - OPTIMIZED ML PIPELINE")
print("=" * 80)

# ============================================================================
# STEP 1: LOAD DATASET
# ============================================================================
print("\n📁 Loading dataset...")

url = "https://drive.google.com/uc?export=download&id=1fd-oGGYVqy5EaqU2wL3dhHEFW-yFXZOa"

try:
    df = pd.read_csv(url)
    print(f"✅ Dataset loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")
except (FileNotFoundError, pd.errors.ParserError) as e:
    print(f"⚠️ Error loading dataset: {e}")
    print("Creating synthetic data for demonstration...\n")

    np.random.seed(42)
    n_samples = 10000

    airlines = ['Southwest', 'Delta', 'American', 'United', 'JetBlue']
    airports = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX', 'CLT', 'MCO']

    df = pd.DataFrame({
        'id': range(1, n_samples + 1),
        'Airline': np.random.choice(airlines, n_samples),
        'Flight': np.random.randint(100, 9999, n_samples),
        'AirportFrom': np.random.choice(airports, n_samples),
        'AirportTo': np.random.choice(airports, n_samples),
        'DayOfWeek': np.random.randint(1, 8, n_samples),
        'Time': np.random.randint(1, 2400, n_samples),
        'Length': np.random.randint(30, 500, n_samples),
        'Delay': np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
    })
    print(f"✅ Synthetic dataset: {df.shape[0]:,} rows")

print("\n📊 First 5 rows:")
print(df.head())

# ============================================================================
# STEP 2: DATA PREPROCESSING
# ============================================================================
print("\n" + "=" * 80)
print("🔍 Data Exploration")
print("=" * 80)

# Find target column
target_col = 'Delay'
if target_col not in df.columns:
    delay_cols = [col for col in df.columns if 'delay' in col.lower()]
    if delay_cols:
        target_col = delay_cols[0]
        print(f"Using '{target_col}' as target variable")
    else:
        raise ValueError("Cannot find delay column!")

print(f"\nTarget distribution:")
print(df[target_col].value_counts())
print(f"Delay rate: {df[target_col].mean():.2%}")
print(f"Missing values: {df.isnull().sum().sum()}")

# Remove ID columns
id_cols = ['id', 'Id', 'ID', 'Flight', 'flight']
df = df.drop(columns=[col for col in id_cols if col in df.columns])

# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col].astype(int)

# Identify feature types
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Removed: # Convert categorical features to 'category' dtype for XGBoost with enable_categorical=True
# Removed: for col in categorical_features:
# Removed:     X[col] = X[col].astype('category')

print(f"\nNumeric features ({len(numeric_features)}): {numeric_features}")
print(f"Categorical features ({len(categorical_features)}): {categorical_features}")

# ============================================================================
# STEP 3: BUILD PREPROCESSING PIPELINE
# ============================================================================
print("\n" + "=" * 80)
print("🏗️ Building preprocessing pipeline")
print("=" * 80)

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))  # Keep sparse for memory
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

# ============================================================================
# STEP 4: TRAIN-VALIDATION-TEST SPLIT
# ============================================================================
print("\n✂️ Splitting data (train: 70%, val: 15%, test: 15%)...")

# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Second split: 50% of temp = 15% val, 15% test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Training: {X_train.shape[0]:,} | Validation: {X_val.shape[0]:,} | Test: {X_test.shape[0]:,}")

# ============================================================================
# STEP 5: DEFINE AND TRAIN MODELS
# ============================================================================
print("\n" + "=" * 80)
print("🤖 Training Models")
print("=" * 80)

models = {
    'Logistic Regression': LogisticRegression(
        max_iter=500,
        random_state=42,
        n_jobs=-1
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100,  # Reduced from 200
        max_depth=10,       # Reduced from 15
        min_samples_split=20,
        random_state=42,
        n_jobs=-1,
        verbose=0
    )
}

# Add XGBoost with GPU if available
if XGB_OK:
    xgb_params = {
        'n_estimators': 100,
        'learning_rate': 0.1,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42,
        'eval_metric': 'logloss',
        'early_stopping_rounds': 10,
        'tree_method': 'gpu_hist' if GPU_AVAILABLE else 'hist'
        # Removed 'enable_categorical': True as OneHotEncoder handles it
    }

    if GPU_AVAILABLE:
        xgb_params['predictor'] = 'gpu_predictor'
        print("🚀 XGBoost configured for GPU acceleration")

    models['XGBoost'] = XGBClassifier(**xgb_params)

results = {}

for name, model in models.items():
    print(f"\n{'─' * 80}")
    print(f"Training: {name}")
    print('─' * 80)

    if name == 'XGBoost' and XGB_OK:
        # For XGBoost with early stopping, manually preprocess eval_set
        # as pipeline.fit(classifier__eval_set) passes raw X_val.

        # 1. Fit the ColumnTransformer (preprocessor) on training data only
        fitted_preprocessor = preprocessor.fit(X_train)

        # 2. Transform training and validation data using the fitted preprocessor
        X_train_transformed = fitted_preprocessor.transform(X_train)
        X_val_transformed = fitted_preprocessor.transform(X_val)

        # 3. Fit the XGBoost classifier directly using transformed data and eval_set
        model.fit(X_train_transformed, y_train,
                  eval_set=[(X_val_transformed, y_val)],
                  verbose=False) # verbose=False to suppress iteration output

        # 4. Construct the final pipeline using the fitted preprocessor and the fitted XGBoost model
        # This pipeline is ready for .predict() on new, raw data.
        pipeline = Pipeline([
            ('preprocessor', fitted_preprocessor),
            ('classifier', model)
        ])
    else:
        # For other models, or if not using early stopping with XGBoost, fit the pipeline normally.
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])
        pipeline.fit(X_train, y_train)

    # Evaluate on validation set
    y_val_pred = pipeline.predict(X_val)
    y_val_proba = pipeline.predict_proba(X_val)[:, 1] if hasattr(model, 'predict_proba') else None

    metrics = {
        'Accuracy': accuracy_score(y_val, y_val_pred),
        'Precision': precision_score(y_val, y_val_pred, zero_division=0),
        'Recall': recall_score(y_val, y_val_pred, zero_division=0),
        'F1': f1_score(y_val, y_val_pred, zero_division=0),
    }

    if y_val_proba is not None:
        metrics['ROC-AUC'] = roc_auc_score(y_val, y_val_proba)

    results[name] = {'pipeline': pipeline, 'metrics': metrics}

    print(f"Validation Results:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name:10s}: {value:.4f}")

# ============================================================================
# STEP 6: SELECT AND EVALUATE BEST MODEL
# ============================================================================
print("\n" + "=" * 80)
print("🏆 Model Selection")
print("=" * 80)

# Compare models by F1 Score (balanced metric for classification)
comparison = pd.DataFrame([
    {'Model': name, **result['metrics']}
    for name, result in results.items()
])
comparison = comparison.sort_values('F1', ascending=False)

print("\nValidation Performance:")
print(comparison.to_string(index=False))

best_model_name = comparison.iloc[0]['Model']
best_pipeline = results[best_model_name]['pipeline']

print(f"\n🥇 Best Model: {best_model_name}")

# Final evaluation on test set
print("\n" + "=" * 80)
print("📊 Final Test Set Evaluation")
print("=" * 80)

y_test_pred = best_pipeline.predict(X_test)
y_test_proba = best_pipeline.predict_proba(X_test)[:, 1] if hasattr(best_pipeline.named_steps['classifier'], 'predict_proba') else None

test_metrics = {
    'Accuracy': accuracy_score(y_test, y_test_pred),
    'Precision': precision_score(y_test, y_test_pred, zero_division=0),
    'Recall': recall_score(y_test, y_test_pred, zero_division=0),
    'F1': f1_score(y_test, y_test_pred, zero_division=0),
}

if y_test_proba is not None:
    test_metrics['ROC-AUC'] = roc_auc_score(y_test, y_test_proba)

print(f"\n{best_model_name} - Test Set Performance:")
for metric, value in test_metrics.items():
    print(f"  {metric:10s}: {value:.4f}")

cm = confusion_matrix(y_test, y_test_pred)
print(f"\nConfusion Matrix:")
print(f"  [[TN={cm[0,0]:5d}, FP={cm[0,1]:5d}]")
print(f"   [FN={cm[1,0]:5d}, TP={cm[1,1]:5d}]]")

# ============================================================================
# STEP 7: SAVE MODEL
# ============================================================================
print("\n" + "=" * 80)
print("💾 Saving Model")
print("=" * 80)

output_path = 'flight_delay_model.pkl'
joblib.dump(best_pipeline, output_path, compress=3)
print(f"✅ Model saved: {output_path}")

model_info = {
    'model_name': best_model_name,
    'test_metrics': test_metrics,
    'features': {
        'numeric': numeric_features,
        'categorical': categorical_features
    },
    'gpu_used': GPU_AVAILABLE if best_model_name == 'XGBoost' else False,
    'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

import json
with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)
print("✅ Model info saved: model_info.json")

# ============================================================================
# STEP 8: EXAMPLE PREDICTION
# ============================================================================
print("\n" + "=" * 80)
print("🧪 Sample Prediction")
print("=" * 80)

sample = pd.DataFrame([{
    'Airline': 'Southwest',
    'AirportFrom': 'LAX',
    'AirportTo': 'JFK',
    'DayOfWeek': 5,
    'Time': 1800,
    'Length': 300
}])

pred = best_pipeline.predict(sample)[0]
proba = best_pipeline.predict_proba(sample)[0] if hasattr(best_pipeline.named_steps['classifier'], 'predict_proba') else None

print(f"\nInput: {sample.to_dict('records')[0]}")
print(f"Prediction: {'⚠️ DELAYED' if pred == 1 else '✅ ON TIME'}")
if proba is not None:
    print(f"Confidence: {proba[pred]:.1%}")

print("\n" + "=" * 80)
print("✅ TRAINING COMPLETE")
print("=" * 80)


⚠️ GPU not available: Invalid Input: 'gpu_hist', valid values are: {'approx', 'auto', 'exact', 'hist'}
   XGBoost will use CPU
FLIGHT DELAY PREDICTION - OPTIMIZED ML PIPELINE

📁 Loading dataset...
✅ Dataset loaded: 539,383 rows, 9 columns

📊 First 5 rows:
   id Airline  Flight AirportFrom AirportTo  DayOfWeek  Time  Length  Delay
0   1      CO     269         SFO       IAH          3    15     205      1
1   2      US    1558         PHX       CLT          3    15     222      1
2   3      AA    2400         LAX       DFW          3    20     165      1
3   4      AA    2466         SFO       DFW          3    20     195      1
4   5      AS     108         ANC       SEA          3    30     202      0

🔍 Data Exploration

Target distribution:
Delay
0    299119
1    240264
Name: count, dtype: int64
Delay rate: 44.54%
Missing values: 0

Numeric features (3): ['DayOfWeek', 'Time', 'Length']
Categorical features (3): ['Airline', 'AirportFrom', 'AirportTo']

🏗️ Building preprocessing pipel